In [58]:
import json
import hmac
import uuid
import hmac
import time
import base64
import hashlib
import datetime
import requests
from urllib.parse import quote
from configparser import ConfigParser

In [60]:
class SignatureUrl():
    """python 计算openapi的签名"""

    def __init__(self, public_param, private_param, secret):
        self.public_param = public_param
        self.private_param = private_param
        self.secret = secret

    def get_timestamp(self):
        time_format = "%Y-%m-%dT%H:%M:%SZ"
        return datetime.datetime.utcnow().strftime(time_format)

    def get_uuid(self):
        return str(uuid.uuid1())

    def url_encode_str(self, all_params):
        sort_all_params = list()
        for key, value in all_params.items():
            params = key + '=' + value
            sort_all_params.append(params)
        # 对参数进行升序排序
        sort_all_params.sort()

        for i in range(len(sort_all_params)):
            # 对参数以及参数值进行urlencode处理,注意:’=‘此时不能处理，否则后面会再次对%3D进行encode
            sort_all_params[i] = quote(sort_all_params[i], '=')
            # 对encode之后的字符串进行再处理
            tmp = sort_all_params[i]
            if tmp.find('+'):
                tmp.replace('+','%20')
            elif tmp.find('*'):
                tmp.replace('*','%2A')
            elif tmp.find('%7E'):
                tmp.replace('%7E','~')
            
            sort_all_params[i] = tmp
        return sort_all_params

    def get_signature(self, param, http_method, AccesskeySecret):
        str_to_sign = ''
        sort_all_params = self.url_encode_str(param)
        #print(sort_all_params)
        for i in range(len(sort_all_params)):
            str_to_sign = str_to_sign + sort_all_params[i] + '&'

        # 将最后一位&给截取掉
        str_to_sign = http_method + '&%2F&' + quote(str_to_sign[:-1])
        #print(str_to_sign)
        key = AccesskeySecret+'&'
        signature = hmac.new(key.encode(
            'utf-8'), str_to_sign.encode('utf-8'), digestmod=hashlib.sha1)
        signature = base64.b64encode(signature.digest()).decode().rstrip("\n")
        # 解决签名中包含有'+'的特殊情况
        signature = list(signature)
        for i in range(len(signature)):
            #signature[i] = str(signature[i])
            if signature[i] == '+':
                signature[i] = '%2B'
        newSignature = ''.join(signature)
        #print ("Signature: " + newSignature)
        self.private_param['Signature'] = newSignature

    def url_factory(self, method):
        all_params = dict(self.public_param, **self.private_param)
        self.get_signature(all_params, method, self.secret)
        url = ''
        par=[]
        for key, value in all_params.items():
            params = key + '=' + value
            par.append(params)
        for i in range(len(par)):
            url = url + par[i] + '&'
        url = 'http://swas.eu-central-1.aliyuncs.com?' + url[:-1] + '&Signature=' + self.private_param['Signature']
        #print('url is : ' + url)
        return url

In [61]:
def config(filename='alibaba_tokens', section='tokens'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    cfg = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            cfg[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return cfg

def upgradeInstance(InstanceId, ImageId, public_param, secret):
    action_param = dict()
    action_param["Action"] = "ResetSystem"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId
    private_param["ImageId"] = ImageId

    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance reset: " + result.text)

def renameInstance(InstanceId, Name, public_param, secret):
    action_param = dict()
    action_param["Action"] = "UpdateInstanceAttribute"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId
    private_param["InstanceName"] = Name

    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance renamed: " + result.text)

In [62]:
params = config()

public_param = dict()
public_param["AccessKeyId"] = params["id"]
public_param["SignatureMethod"] = 'HMAC-SHA1'
public_param["SignatureVersion"] = '1.0'
public_param["Version"] = "2020-06-01"
public_param["Format"] = 'json'

action_param = dict()
action_param["Action"] = "ListInstances"

instances = []
baseInstance = "9d4c9fd55d884badba2540b561432c1e"

j=0
for i in range(3):
    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["RegionId"] = "eu-central-1"
    private_param["PageSize"] = "100"
    private_param["PageNumber"] = str(i+1)

    sig = SignatureUrl(public_param, private_param, params["secret"])
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')
    result = requests.request(method="get",url=url)
    for instance in json.loads(result.text)["Instances"]:
        j += 1
        print(str(j) + ". " + instance["InstanceId"] + " has ImageId " + instance["InstanceId"])
        time.sleep(0.5)
        if instance["InstanceId"] != baseInstance:
            upgradeInstance(instance["InstanceId"], "m-gw81z641z0hnm4fr5zd8", public_param, params["secret"])
            renameInstance(instance["InstanceId"], "cah_v6_2_" + str(j), public_param, params["secret"])
            time.sleep(15)
        

1. 678114befd274c71bf5393dc6b6f6550 has ImageId 678114befd274c71bf5393dc6b6f6550
Instance reset: {"RequestId":"DFD01755-1833-3469-A71A-513C120970AC"}
Instance renamed: {"RequestId":"A5F45D50-5500-3A31-B793-34B7B587436A"}
2. 14091f7a77ea49bab8a13cc141931f86 has ImageId 14091f7a77ea49bab8a13cc141931f86
Instance reset: {"RequestId":"0B9AFAC7-A672-371B-812B-C88F19D7CA2C"}
Instance renamed: {"RequestId":"3149C017-7CED-334E-BFB8-8E7D42EAEDF5"}


KeyboardInterrupt: 